### 3.1 计算图

tensorflow是一个通过计算图的形式来表述计算的编程系统。每一个计算都是计算图上的一个节点，节点之间的边描述了计算之间的依赖关系。

程序一般分为两个阶段：

第一阶段：定义计算图中所有的计算；

第二阶段：执行计算。

In [5]:
import tensorflow as tf
a = tf.constant([1.0, 2.0], name="a")
b = tf.constant([2.0, 3.0], name="b")
result = a + b
print(result)

# 通过 a.graph 可以查看张量所属的计算图
print(a.graph)

# 因为没有特意指定，所以这个计算图应该等于当前默认的计算图
# tf.get_default_graph() 获取当前默认的计算图
print(a.graph is tf.get_default_graph())

Tensor("add_4:0", shape=(2,), dtype=float32)
True


tf.Graph 生成新的计算图。不同计算图上的张量和运算都不会共享。

示例：在不同计算图上定义和使用变量

In [7]:
import tensorflow as tf

g1 = tf.Graph()
with g1.as_default():
    # 在计算图 g1 上定义变量 “v”, 并设置初始值为0
    v = tf.get_variable("v", [1], initializer = tf.zeros_initializer()) 

g2 = tf.Graph()
with g2.as_default():
    # 在计算图 g2 上定义变量 “v”, 并设置初始值为1
    v = tf.get_variable("v", [1], initializer = tf.ones_initializer())  

# 在计算图 g1 中读取变量 “v” 的值
with tf.Session(graph = g1) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable("v")))

# 在计算图 g2 中读取变量 “v” 的值
with tf.Session(graph = g2) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable("v")))
        

[0.]
[1.]


计算图不仅可以用来隔离张量和计算，还提供了管理张量和计算的机制。

计算图可以通过 tf.Graph.device 函数来指定运行计算的设备。

```
g = tf.Graph()
# 指定运行的设备
with g.device('/gpu:0'):
    result = a + b
```

In [2]:
sess = tf.InteractiveSession() 
v = tf.Variable([1,2], dtype=tf.float32)
sess.run(tf.global_variables_initializer())
print(v.eval())
sess.close()

[ 1.  2.]


### 3.2 张量
所有的数据都通过张量的形式来表示。张量在功能上可以理解为多维数组。（注：张量的类型也可以是字符串）

在tf中张量的实现并不是直接采用数组的形式，它只是对tf中运算结果的引用。在张量中，并没有真正保存数字，它保存的是如何得到这些数字的计算过程。

In [9]:
import tensorflow as tf
# tf.constant 是一个计算，这个计算的结果为一个张量
a = tf.constant([1.0, 2.0], name="a")
b = tf.constant([2.0, 3.0], name="b")
result = a + b
print("result:", result)

result: Tensor("add_6:0", shape=(2,), dtype=float32)


以上可以看出，tf计算出的结果是一个张量的结构，主要保存了三个属性：name, shape, type

name: 张量的唯一标识符，同时给出该张量是如何计算出来的；

shape: 描述了张量的维度信息；

type: 张量的唯一类型。

### 3.3 会话
执行定义好的运算。拥有并管理tf运行时的所有资源。

In [10]:
# 交互环境下直接构建默认会话，省去将产生的会话注册为默认会话的过程
sess = tf.InteractiveSession ()
print("result.eval()", result.eval())
sess.close()

result.eval() [3. 5.]


In [11]:
# 创建一个会话。
sess = tf.Session()

# 使用会话得到之前计算的结果。
print(sess.run(result))

# 关闭会话使得本次运行中使用到的资源可以被释放。
sess.close()

[3. 5.]


In [5]:
with tf.Session() as sess:
    print(sess.run(result))

[ 3.  5.]


In [12]:
# 指定默认会话
sess = tf.Session()
with sess.as_default():
     print(result.eval())



[3. 5.]


In [7]:
sess = tf.Session()

# 下面的两个命令有相同的功能。
print(sess.run(result))
print(result.eval(session=sess))

[ 3.  5.]
[ 3.  5.]


通过configProto配置参数

allow_soft_placement: bool, 默认为false。在有GPU的环境中，一般设置为true。当某些运算无法被当前GPU支持时，可以自动调整到CPU上，而不是报错。

log_device_placement: bool。当为true时，日志中将积累每个节点被安排在哪个设备上方便调试。

In [9]:
config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)
sess1 = tf.InteractiveSession(config=config)
sess2 = tf.Session(config=config)